In [ ]:
!pip install matplotlib seaborn


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Cài đặt cho đồ thị đẹp hơn
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Các thư viện đã sẵn sàng!")

Các thư viện đã sẵn sàng!


In [ ]:
# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
db_user = os.getenv("MYSQL_USER")
db_password = os.getenv("MYSQL_PASSWORD")
db_host = os.getenv("MYSQL_HOST")
db_port = os.getenv("MYSQL_PORT")
db_name = os.getenv("MYSQL_DATABASE")

# Tạo chuỗi kết nối và engine
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

# Viết câu truy vấn SQL để lấy toàn bộ dữ liệu
query = "SELECT * FROM transactions"

# Đọc dữ liệu từ SQL vào DataFrame của pandas
df = pd.read_sql(query, engine)

# Kiểm tra xem dữ liệu đã được tải đúng chưa
print(f"Đã tải thành công {len(df)} dòng dữ liệu.")
df.info()
df.head()

In [ ]:
# Tính toán cột doanh thu
df['revenue'] = df['quantity'] * df['unitprice']

# Chuyển invoicedate thành index để dễ dàng resample
df_time = df.set_index('invoicedate')

# Resample dữ liệu theo tháng và tính tổng doanh thu
monthly_revenue = df_time['revenue'].resample('M').sum()

# Vẽ biểu đồ
plt.figure(figsize=(15, 7))
monthly_revenue.plot(title='Doanh thu hàng tháng', marker='o')
plt.xlabel('Tháng')
plt.ylabel('Tổng Doanh thu')
plt.show()

In [ ]:
# Nhóm theo quốc gia và tính tổng doanh thu
country_revenue = df.groupby('country')['revenue'].sum().sort_values(ascending=False)

# Lấy top 10 quốc gia
top_10_countries = country_revenue.head(10)

# Vẽ biểu đồ cột
plt.figure(figsize=(15, 7))
sns.barplot(x=top_10_countries.index, y=top_10_countries.values, palette='viridis')
plt.title('Top 10 quốc gia có doanh thu cao nhất')
plt.xlabel('Quốc gia')
plt.ylabel('Tổng Doanh thu')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Nhóm theo ID khách hàng và tính tổng chi tiêu
customer_spending = df.groupby('customerid')['revenue'].sum().sort_values(ascending=False)

# Lấy top 10 khách hàng
top_10_customers = customer_spending.head(10)

# In ra top 10 khách hàng
print("Top 10 khách hàng chi tiêu nhiều nhất:")
print(top_10_customers)

# Vẽ biểu đồ
plt.figure(figsize=(15, 7))
sns.barplot(x=top_10_customers.index.astype(str), y=top_10_customers.values, palette='plasma', order=top_10_customers.index.astype(str))
plt.title('Top 10 khách hàng chi tiêu nhiều nhất')
plt.xlabel('Customer ID')
plt.ylabel('Tổng chi tiêu')
plt.show()

In [ ]:
# Nhóm theo mã sản phẩm và mô tả, tính tổng số lượng bán ra
product_sales = df.groupby(['stockcode', 'description'])['quantity'].sum().sort_values(ascending=False)

# Lấy top 10 sản phẩm
top_10_products = product_sales.head(10)

# In ra top 10 sản phẩm
print("Top 10 sản phẩm bán chạy nhất:")
print(top_10_products)

# Vẽ biểu đồ
plt.figure(figsize=(15, 7))
sns.barplot(y=top_10_products.index.get_level_values('description'), x=top_10_products.values, palette='magma')
plt.title('Top 10 sản phẩm bán chạy nhất')
plt.xlabel('Tổng số lượng đã bán')
plt.ylabel('Sản phẩm')
plt.show()

In [ ]:
Tóm tắt các insight chính:

Doanh thu có xu hướng tăng mạnh vào cuối năm, đặc biệt là tháng 11.

Thị trường Vương quốc Anh là thị trường cốt lõi, chiếm phần lớn doanh thu.

Có một nhóm khách hàng VIP chi tiêu cao hơn hẳn so với phần còn lại.

Xác định được các sản phẩm chủ lực cần tập trung.